In [1]:
import pandas as pd
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
    RobertaTokenizer,
    RobertaForMaskedLM,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
)
import os, json, torch
import pandas as pd
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
import time 
import random


In [20]:
a = pd.read_csv("./train.csv")

questionn = a['question']
context = a["context"]

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [45]:

def rotate_space_col(input_data, tokenizer, col='context'):
    origin_data = input_data.copy()

    rotated_con_lst = []
    for con in tqdm(origin_data[col].tolist()):
        splitted_con = con.strip().split(".")
        seq_len = len(splitted_con)
        print(seq_len)
        print(splitted_con)
        slicing_idx = random.randrange(1, seq_len)
        rotated_sentence = '. '.join(splitted_con[slicing_idx:] + splitted_con[:slicing_idx])

        rotated_con_lst.append(rotated_sentence.strip())

    origin_data[col] = rotated_con_lst
    return origin_data


In [46]:
 a= rotate_space_col(input_data = a[0:3], tokenizer = tokenizer)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 5580.01it/s]

4
['\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다', ' 날짜=2017-02-05', ' 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다', ' \\n\\n미국 부통령이 상원의장이 된다 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다']
3
["\\n\\n현대적 인사조직관리와 근대 인사조직관리의 가장 큰 차이는 통합이다 19세기의 영향을 받던 근대적 경영학(고전적 경영)의 흐름은 기능을 강조하였지만, 1950년대 이후의 현대 경영학은 통합을 강조하였다 기능이 

In [47]:

for i in a['context']:
    print(i)

날짜=2017-02-05.  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.  \n\n미국 부통령이 상원의장이 된다 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. \n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다
'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 1950년대이다 2차 세계대전을 마치고, 6.  25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 아프리카, 아시아, 아메리카 대륙에서 신생국가가